In [1]:
import pandas as pd
import numpy as np

In [2]:
path_to_players = "../data/league_players.csv"
players = pd.read_csv(path_to_players)

In [74]:
def game_day_performance(players_df, stat):
    gdp = np.array(
        [
            np.random.normal(
                loc=players_df.loc[i][f"{stat}_mean"],
                scale=players_df.loc[i][f"{stat}_std"]
            ) for i in players_df.index.to_list()
        ]
    )
    gdp[gdp < 0] = 0
    return gdp

def normalize_performance(gdp):
    if np.sum(gdp) == 0:
        return np.ones_like(gdp)/len(gdp)
    return gdp/np.sum(gdp)

def simulate_a_game(date, teams, players_df):
    # Load goalies
    print(f"Simulating a game on {date} between the {teams[0]} and {teams[1]}")
    players_by_team = [
        players_df.loc[players_df["team"] == team] for team in teams
    ]
    
    game_report_df = dict()
    game_report_df["name"] = []
    game_report_df["team"] = []
    game_report_df["position"] = []
    for team_players in players_by_team:
        game_report_df["name"].extend(team_players["name"].to_list())
        game_report_df["team"].extend(team_players["team"].to_list())
        game_report_df["position"].extend(team_players["position"].to_list())
    
    game_report_df = pd.DataFrame(game_report_df)
    game_report_df["Goals"] = 0
    game_report_df["Assists"] = 0
    game_report_df["Hits"] = 0
    game_report_df["Blocks"] = 0
    game_report_df["Game_Starts"] = 0
    game_report_df["Wins"] = 0
    game_report_df["Saves"] = 0
    
    # Choose a starter
    goalies_by_team = [
        team_players.loc[team_players["position"] == "G"] for team_players in players_by_team
    ]
    starters_by_team = []
    for goalies in goalies_by_team:
        starting_proba = normalize_performance(game_day_performance(goalies, "Game_Starts"))
        starters_by_team.append(
            np.random.choice(
                goalies.index.to_list(),
                p=starting_proba
            )
        )
    
        
    starter_names_by_team = players_df.loc[starters_by_team]["name"].to_list()
    print(f"\tStarting goaltender in tonight's game: {starter_names_by_team[0]} and {starter_names_by_team[1]}")
    game_report_df.loc[game_report_df["name"].isin(starter_names_by_team), "Game_Starts"] = 1
    
    # Flip a coin to chose the winning goalie
    win_performances = [
        max(
            np.random.normal(
                loc=players_df.loc[starter_index]["Wins_mean"],
                scale=players_df.loc[starter_index]["Wins_std"]
            ), 
            0
        ) for starter_index in starters_by_team
    ]
    
    winning_probabilities = normalize_performance(np.array(win_performances))
    winner = np.random.choice(starter_names_by_team, p=winning_probabilities)
    game_report_df.loc[game_report_df["name"] == winner, "Wins"] = 1
    winning_index = starter_names_by_team.index(winner)
    losing_index = int(np.abs(winning_index - 1)) #returns 0 if winning is 1, 1 if winning is 0
    
    # Randomly generate a score
    final_score = np.abs(np.random.normal(loc=0, scale=4, size=2)).astype(int)
    while final_score[winning_index] <= final_score[losing_index]:
        final_score = np.abs(np.random.normal(loc=0, scale=4, size=2)).astype(int)
    print(f"\tFinal Score {final_score[0]} - {final_score[1]}")
    
    # Randomly select which goals get assist and how many
    final_assists = [np.random.randint(low=0, high=3, size=goals) for goals in final_score]
    
    # Subselect players for goals by randomly sampling a "performance"
    skaters_by_team = [
        team_players.loc[team_players["position"] != "G"] for team_players in players_by_team
    ]
    
    goals_perf = [
        normalize_performance(game_day_performance(team_players, "Goals")) for team_players in skaters_by_team
    ]
    
    assists_perf = [
        game_day_performance(team_players, "Assists") for team_players in skaters_by_team
    ]
    
    for i, score in enumerate(final_score):
        print(f"\tGoal Scoring for {teams[i]}:")
        skaters = skaters_by_team[i]
        assists = final_assists[i]
        
        #choose the goal scorers
        goals_scorers = np.random.choice(
            skaters["name"].to_list(),
            size=score,
            replace=True,
            p=goals_perf[i]
        )
        
        # choose the assisting players
        for j, assist in enumerate(assists):
            goal_scorer = goals_scorers[j]
            print(f"\t\tScored by:{goal_scorer}")
            game_report_df.loc[game_report_df["name"] == goal_scorer, "Goals"] += 1
            
            remaining_skaters = skaters.loc[skaters["name"] != goal_scorer]
            goal_scorer_idx = skaters["name"].to_list().index(goal_scorer)
            remaining_assists_perf = normalize_performance(
                assists_perf[i][:goal_scorer_idx].tolist() + assists_perf[i][goal_scorer_idx+1:].tolist()
            )
            assist_scorers = np.random.choice(
                remaining_skaters["name"].to_list(),
                size=assists[j],
                replace=False,
                p=remaining_assists_perf
            )
        
            for assist in assist_scorers:
                print(f"\t\t\tassisted by {assist}")
                game_report_df.loc[game_report_df["name"] == assist, "Assists"] += 1 
            
        game_report_df.loc[
            game_report_df["name"].isin(skaters["name"].to_list()), 
            "Hits"] = game_day_performance(skaters, "Hits").astype(int)
        game_report_df.loc[
            game_report_df["name"].isin(skaters["name"].to_list()), 
            "Blocks"] = game_day_performance(skaters, "Blocks").astype(int)
        # Simulate saves per goalie
        starter_index = starters_by_team[i]
        saves_by_team = max(
            int(np.random.normal(
                loc=players_df.loc[starter_index]["Saves_mean"],
                scale=players_df.loc[starter_index]["Saves_std"],
            )), 0)
        game_report_df.loc[game_report_df["name"] == starter_names_by_team[i], "Saves"] = saves_by_team
        print(f"\tSaves made by {starter_names_by_team[i]}: {saves_by_team}")
    # Return game report
    game_report_df["date"] = date
    return final_score, game_report_df

In [76]:
for season in range(5):
    print("Simulating season", season)
    year = 2015 + season
    path_to_schedule = f"../data/schedules/season_{season}_{year}-{year+1}.csv"
    schedule = pd.read_csv(path_to_schedule)

    scores = []
    for index, row in schedule.iterrows():
        d = row["date"]
        t = row[["team_1","team_2"]].to_list()
        score, report = simulate_a_game(d, t, players)
        report.to_csv(f"../data/game_reports/season_{season}/{t[0]}_{t[1]}_{d}_game_report.csv")
        scores.append((d, t, score))

Simulating season 0
Simulating a game on 2015-12-12 between the Hat-Trick Heroes and No Woman, No Krejci
	Starting goaltender in tonight's game: Timothy Johnson and Kelly Snyder
	Final Score 4 - 5
	Goal Scoring for Hat-Trick Heroes:
		Scored by:Amanda Parker
			assisted by Jeffrey Green
		Scored by:Danielle Perez
			assisted by Kimberly Hart
			assisted by Karen Smith
		Scored by:David Bradley
			assisted by Karen Smith
		Scored by:Michele Herman
	Saves made by Timothy Johnson: 38
	Goal Scoring for No Woman, No Krejci:
		Scored by:Steven Sanchez MD
			assisted by Kelly Meadows
		Scored by:Mackenzie Davis
		Scored by:Carolyn Moore
			assisted by Kelly Meadows
		Scored by:Joseph Morris
		Scored by:Carolyn Moore
			assisted by Regina Thompson
	Saves made by Kelly Snyder: 47
Simulating a game on 2016-01-18 between the Hat-Trick Heroes and Fresh Prince of Briere
	Starting goaltender in tonight's game: Elizabeth Smith and Amber Rivera
	Final Score 3 - 0
	Goal Scoring for Hat-Trick Heroes:
		

	Saves made by Timothy Johnson: 37
	Goal Scoring for Fresh Prince of Briere:
		Scored by:Nicole Johnson
			assisted by Jason Clarke MD
			assisted by Joshua Morse
		Scored by:Anthony Richard
		Scored by:Jonathan Torres
			assisted by Mrs. Tanya Hansen
			assisted by Joshua Morse
		Scored by:Valerie Romero
		Scored by:Jason Clarke MD
			assisted by Jonathan Torres
			assisted by Joshua Morse
		Scored by:Ricky Boyd
			assisted by Kevin Gonzalez
			assisted by Joshua Morse
	Saves made by Amber Rivera: 42
Simulating a game on 2016-01-12 between the Hat-Trick Heroes and Flying Elbows
	Starting goaltender in tonight's game: Timothy Johnson and Audrey Garcia
	Final Score 5 - 6
	Goal Scoring for Hat-Trick Heroes:
		Scored by:Karen Smith
			assisted by Jerry Miles
		Scored by:Kyle Smith II
			assisted by John Jordan
			assisted by Danielle Perez
		Scored by:Danielle Perez
		Scored by:Kyle Smith II
			assisted by Jerry Miles
		Scored by:Jerry Miles
			assisted by John Jordan
	Saves made by Timot

			assisted by Anthony Richard
		Scored by:Logan Harmon
		Scored by:Mrs. Tanya Hansen
			assisted by Joshua Morse
			assisted by Dennis Irwin
	Saves made by Amber Rivera: 33
Simulating a game on 2015-10-14 between the Hat-Trick Heroes and No Woman, No Krejci
	Starting goaltender in tonight's game: Elizabeth Smith and Kelly Snyder
	Final Score 5 - 1
	Goal Scoring for Hat-Trick Heroes:
		Scored by:Nicholas Jones
		Scored by:Phillip Kaiser
		Scored by:Kyle Smith II
			assisted by Phillip Kaiser
		Scored by:Nicholas Jones
		Scored by:Danielle Perez
	Saves made by Elizabeth Smith: 43
	Goal Scoring for No Woman, No Krejci:
		Scored by:Steven Sanchez MD
	Saves made by Kelly Snyder: 25
Simulating a game on 2016-01-31 between the No Woman, No Krejci and Flying Elbows
	Starting goaltender in tonight's game: Kelly Snyder and Kevin Hubbard
	Final Score 5 - 6
	Goal Scoring for No Woman, No Krejci:
		Scored by:Michael Buck
			assisted by Linda Dalton
			assisted by Joseph Morris
		Scored by:Megan Ro

	Saves made by Kelly Snyder: 36
	Goal Scoring for Fresh Prince of Briere:
		Scored by:Anthony Richard
		Scored by:Kevin Gonzalez
		Scored by:Eric Young
	Saves made by Amber Rivera: 16
Simulating a game on 2015-12-22 between the No Woman, No Krejci and Flying Elbows
	Starting goaltender in tonight's game: Kelly Snyder and Jason Robinson
	Final Score 5 - 8
	Goal Scoring for No Woman, No Krejci:
		Scored by:Joseph Morris
			assisted by Regina Thompson
			assisted by Carolyn Moore
		Scored by:Joseph Morris
		Scored by:Kelly Meadows
			assisted by Steven Sanchez MD
		Scored by:Michael Buck
		Scored by:Kelly Meadows
	Saves made by Kelly Snyder: 40
	Goal Scoring for Flying Elbows:
		Scored by:Danny Mendoza
		Scored by:Anita Jones
		Scored by:Michael Blake
			assisted by April Andrews
			assisted by Todd Howe
		Scored by:Todd Howe
		Scored by:Christopher Jones
			assisted by April Andrews
		Scored by:Michael Blake
		Scored by:Christopher Jones
		Scored by:Marissa Larson
			assisted by April An

	Saves made by Ruth Brandt: 34
Simulating a game on 2016-05-23 between the Flying Elbows and Fresh Prince of Briere
	Starting goaltender in tonight's game: Jason Robinson and Christopher Rodriguez
	Final Score 0 - 1
	Goal Scoring for Flying Elbows:
	Saves made by Jason Robinson: 17
	Goal Scoring for Fresh Prince of Briere:
		Scored by:Kevin Gonzalez
			assisted by Joshua Morse
	Saves made by Christopher Rodriguez: 27
Simulating a game on 2015-12-03 between the Flying Elbows and Fresh Prince of Briere
	Starting goaltender in tonight's game: Jason Robinson and Christopher Rodriguez
	Final Score 1 - 4
	Goal Scoring for Flying Elbows:
		Scored by:Christopher Jones
	Saves made by Jason Robinson: 22
	Goal Scoring for Fresh Prince of Briere:
		Scored by:Mrs. Tanya Hansen
			assisted by Eric Young
			assisted by Jason Clarke MD
		Scored by:Ricky Boyd
		Scored by:Eric Young
			assisted by Nicole Johnson
			assisted by Anthony Richard
		Scored by:Kevin Gonzalez
	Saves made by Christopher Rodrigu

...

	Saves made by Erika Alexander: 61
Simulating a game on 2020-05-10 between the No Woman, No Krejci and Flying Elbows
	Starting goaltender in tonight's game: Kelly Snyder and Kevin Hubbard
	Final Score 2 - 5
	Goal Scoring for No Woman, No Krejci:
		Scored by:Angel Moon
			assisted by Mackenzie Davis
		Scored by:Michael Buck
			assisted by Regina Thompson
	Saves made by Kelly Snyder: 32
	Goal Scoring for Flying Elbows:
		Scored by:Anita Jones
		Scored by:Heather Lambert
			assisted by Jason Williams
		Scored by:April Andrews
			assisted by Christopher Jones
		Scored by:Michael Blake
			assisted by Chad Schwartz
		Scored by:Heather Lambert
			assisted by Danny Mendoza
			assisted by Chad Schwartz
	Saves made by Kevin Hubbard: 87
Simulating a game on 2020-04-14 between the No Woman, No Krejci and The Lucky Pucks
	Starting goaltender in tonight's game: Kelly Snyder and Ruth Brandt
	Final Score 3 - 4
	Goal Scoring for No Woman, No Krejci:
		Scored by:Joseph Morris
		Scored by:Steven Sanchez 

	Goal Scoring for Flying Elbows:
		Scored by:April Andrews
			assisted by Todd Howe
			assisted by Chad Schwartz
		Scored by:Michael Blake
			assisted by Chad Schwartz
		Scored by:Jason Williams
			assisted by Heather Lambert
	Saves made by Audrey Garcia: 12
	Goal Scoring for Benchwarmers:
	Saves made by Erika Alexander: 59
Simulating a game on 2020-02-01 between the Flying Elbows and Benchwarmers
	Starting goaltender in tonight's game: Audrey Garcia and Erika Alexander
	Final Score 1 - 3
	Goal Scoring for Flying Elbows:
		Scored by:Michael Blake
	Saves made by Audrey Garcia: 21
	Goal Scoring for Benchwarmers:
		Scored by:Kayla Flores
		Scored by:Kayla Flores
		Scored by:Angel Wilkinson
			assisted by Richard Rowland
			assisted by Danielle Harris
	Saves made by Erika Alexander: 59
Simulating a game on 2019-11-19 between the Flying Elbows and PCUK
	Starting goaltender in tonight's game: Jason Robinson and Brandon Schultz
	Final Score 1 - 0
	Goal Scoring for Flying Elbows:
		Scored by:J

	Saves made by Christopher Rodriguez: 8
	Goal Scoring for Benchwarmers:
		Scored by:Teresa Boyd
		Scored by:Andrea Melendez
			assisted by Mark Vincent
		Scored by:Danielle Harris
			assisted by Victor Coleman
			assisted by Richard Rowland
	Saves made by Caleb Miller: 2
Simulating a game on 2020-03-29 between the Fresh Prince of Briere and PCUK
	Starting goaltender in tonight's game: Christopher Rodriguez and Elizabeth York
	Final Score 3 - 2
	Goal Scoring for Fresh Prince of Briere:
		Scored by:Logan Harmon
		Scored by:Joshua Morse
			assisted by Dennis Irwin
		Scored by:Jonathan Torres
			assisted by Ricky Boyd
	Saves made by Christopher Rodriguez: 15
	Goal Scoring for PCUK:
		Scored by:Michelle Miller
			assisted by Jennifer Swanson
			assisted by Sherry White
		Scored by:Jennifer Swanson
	Saves made by Elizabeth York: 2
Simulating a game on 2020-05-15 between the Fresh Prince of Briere and Blades of Steel
	Starting goaltender in tonight's game: Amber Rivera and Amy Cook
	Final Sco

			assisted by Bryan Hayes
	Saves made by Ruth Brandt: 50
	Goal Scoring for Benchwarmers:
		Scored by:Alicia Ward
		Scored by:Kayla Flores
	Saves made by Erika Alexander: 59
Simulating a game on 2020-04-28 between the The Lucky Pucks and PCUK
	Starting goaltender in tonight's game: Ruth Brandt and Elizabeth York
	Final Score 10 - 0
	Goal Scoring for The Lucky Pucks:
		Scored by:Lauren Grant
			assisted by Rhonda Miller
			assisted by Kayla Matthews
		Scored by:Whitney Owens
			assisted by Kayla Matthews
		Scored by:Victor Garcia
			assisted by Whitney Owens
			assisted by Bryan Hayes
		Scored by:Rhonda Miller
			assisted by Juan Walton
			assisted by Kayla Matthews
		Scored by:Lauren Grant
			assisted by Steven Peters
		Scored by:Juan Hamilton
			assisted by Whitney Owens
			assisted by Kayla Matthews
		Scored by:Whitney Owens
			assisted by Kayla Matthews
		Scored by:Lauren Grant
		Scored by:Lauren Grant
		Scored by:Juan Hamilton
			assisted by Rhonda Miller
	Saves made by Ruth Brandt

			assisted by Cynthia Knight
	Saves made by Brandon Schultz: 60
	Goal Scoring for Blades of Steel:
		Scored by:Michelle Hamilton
		Scored by:Rachel Martinez
			assisted by Shannon Gaines
			assisted by Valerie Jones
	Saves made by Melissa Green: 7
Simulating a game on 2020-01-04 between the PCUK and Benchwarmers
	Starting goaltender in tonight's game: Elizabeth York and Erika Alexander
	Final Score 0 - 6
	Goal Scoring for PCUK:
	Saves made by Elizabeth York: 4
	Goal Scoring for Benchwarmers:
		Scored by:Andrea Melendez
			assisted by Richard Rowland
		Scored by:Kayla Flores
			assisted by Victor Coleman
		Scored by:Andrea Melendez
			assisted by Nathan Johnson
			assisted by Victor Coleman
		Scored by:Andrea Melendez
			assisted by Victor Coleman
		Scored by:Danielle Harris
			assisted by Teresa Boyd
			assisted by Richard Rowland
		Scored by:Victor Coleman
			assisted by Christina Perez
			assisted by Joshua Atkinson
	Saves made by Erika Alexander: 59
Simulating a game on 2020-05-18 

In [9]:
np.random.choice([1, 2, 3], size=0)

array([], dtype=int64)